En este notebook se busca extraer la información necesaria para ser consumida por los modelos.

Variables a considerar:
- Aportes hídricos
- Compras en bolsa
- Compras en contratos
- Demanda del SIN
- Disponibilidad Declarada
- Disponibilidad Real
- Generación real
- Indice ONI
- Precio de bolsa
- Precio de oferta
- Reservas hídricas
- Ventas en bolsa
- Ventas en contratos
- Vertimientos


In [1]:
!pip install pydataxm
# !pip list

In [2]:
import datetime as dt
import pydataxm
from pydataxm.pydataxm import ReadDB
import pandas as pd
import pathlib
import os
import time
# os.chdir(pathlib.Path.cwd())

#Conexión de google drive con el repo
from google.colab import drive
drive.mount('/content/drive')

###==========================
ruta = os.path.join("/content/drive/MyDrive", "MonografiaUdeA")
os.chdir(ruta)
# print("Ruta:", ruta)
abspath = pathlib.Path.cwd()
print(f"Directorio de trabajo: {abspath}")
from periods import get_periods
###==========================

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directorio de trabajo: /content/drive/MyDrive/MonografiaUdeA


In [3]:
# !git status

In [4]:
consult = ReadDB()
consult.get_collections().sort_values(by = 'MetricName').to_csv(os.path.join(abspath, "metrics.csv"), index=False)

## Parámetros para la extracción de la información

In [1]:
start_date = dt.date(2010, 1, 1)
# end_date = dt.date(2000,1,31)
end_date = dt.date(2019, 12, 31)

months_per_period = 12
periods = get_periods(start_date, end_date, months_per_period)


dataset_folder_name = "datasets"
metricas = [
    # ["AporEner","Sistema"],
    # ["CompBolsNaciEner", "Sistema"],
    # ["CompContEner", "Sistema"],
    # ["DemaReal","Sistema"],
    # ["DispoDeclarada", "Recurso"],
    # ["DispoReal", "Recurso"],
    # ["Gene", "Recurso"],
    # ["PrecBolsNaci", "Sistema"],
    # ["PrecOferDesp", "Recurso"],
    # ["VentBolsNaciEner", "Sistema"],
    # ["VentContEner", "Sistema"],
    # ["VoluUtilDiarEner", "Sistema"],
    # ["VertEner", "Sistema"]
]

NameError: name 'dt' is not defined

## Obtener listado de los recursos de generación

In [6]:
recursos_generacion = consult.request_data(
                                          "ListadoRecursos",
                                          "Sistema",
                                          start_date,
                                          end_date
                                          )

recursos_generacion.to_csv(os.path.join(abspath, dataset_folder_name, "listado_recursos.csv"), index=False)
recursos_generacion.head()

,Id,Values_Code,Values_Name,Values_Type,Values_Disp,Values_RecType,Values_CompanyCode,Values_EnerSource,Values_OperStartdate,Values_State,Date
0,Sistema,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,2024-05-13
1,Sistema,2QEK,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,2024-05-13
2,Sistema,2QRL,LA REBUSCA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,GEN. DISTRIBUIDA,HZEG,AGUA,2014-07-24,OPERACION,2024-05-13
3,Sistema,2QV2,BAJO TULUA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,EPSG,AGUA,2015-01-30,OPERACION,2024-05-13
4,Sistema,2R22,LAGUNETA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,ENDG,AGUA,2014-12-17,OPERACION,2024-05-13


In [10]:
if not os.path.exists(os.path.join(abspath, dataset_folder_name)):
    os.makedirs(os.path.join(abspath, dataset_folder_name))

def extraer():
  for metrica in metricas:
      consult = ReadDB()
      df = pd.DataFrame()
      print(metrica)
      # metrica = _reg
      for period in periods:
          print(period[0], period[1])
          df_cons = consult.request_data(metrica[0], metrica[1], period[0], period[1])
          df = pd.concat([df, df_cons])
          time.sleep(2)

      df.to_csv(os.path.join(abspath, dataset_folder_name, f"{metrica[0]}.csv"), index=False)

extraer()

['DemaReal', 'Sistema']
2010-01-01 2010-12-31
2011-01-01 2011-12-31
2012-01-01 2012-12-31
2013-01-01 2013-12-31
2014-01-01 2014-12-31
2015-01-01 2015-12-31
2016-01-01 2016-12-31
2017-01-01 2017-12-31
2018-01-01 2018-12-31
2019-01-01 2019-12-31
